# Long range interaction

Computation of long range phase shifts on the lattice.

This notebook shall identify a local potential with long range components (non-zero effective range but sparse in coordinate or momentum space).
Preferably, the result should be cutoff independent.
For now, $S$-wave results are sufficinet.
The current idea is to choose a potential which mimics chiral LO interactions (contact + one pion exchange) and numerically compute the phase shifts with high precision.

$$\renewcommand{\vec}[1]{\boldsymbol{#1}}$$

---
**Notes:** I first analyzed a separable potential $V(p, p') = - g^*(p) g(p')$ but this is very expensive to compute on the lattice (dense Hamiltonian).


## Numerical compututation of phase shifts

This section provides a numerical extraction of phase shifts.

### Init

Modules:

In [ ]:
import numpy as np
from itertools import product
import matplotlib.pylab as plt

from opdens.utilities.mesh import trns

Parameters (in MeV)

In [ ]:
HBARC = 197
mN = 939 /HBARC
mu = mN / 2

Momentum grid points:

In [ ]:
p, wp = trns(40, 20)

p = np.linspace(1.e-6, 20, 200)
wp = np.ones(p.size) * (p[1] - p[0])

H0 = np.diag(p**2 / 2 / mu)

In [ ]:
1 / pp0**2

In [ ]:
plt.plot(p, 2 * mu * wp / (pp0**2 - p**2), "o")

In [ ]:
#p, wp = trns(100, 100)
pp0 =  15


gint = np.sum(2 * mu * wp / (pp0**2 - p**2 ))

print(gint, np.log((p[-1] + pp0) / (p[-1] - pp0)) * mu / pp0 )
print(np.log((p[-1] + pp0) / (p[-1] - pp0)) * mu / pp0 - gint)

### Idea

Compute the $T$-matrix numerically on a partial wave grid $\left| k, \alpha\right\rangle$

\begin{align}
    T_{\alpha'\alpha}\left(k^{\prime}, k ; E\right)
    &=
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    +
    \sum_{\alpha''}
    \int d q q^{2} 
    \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}+i \epsilon}
    \\
    &=
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    +
    \sum_{\alpha''}\mathcal P \int_{0}^{\infty} d q q^{2} 
        \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}}
    -  i \pi \sum_{\alpha''}\int_{0}^{\infty} d q q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \delta(E-E_{q})
    \\
    &=
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    +
    \sum_{\alpha''}\mathcal P \int_{0}^{\infty} d q q^{2} 
        \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}}
    -  i \pi \sum_{\alpha''} \left[ \mu q \, V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \right]_{q = \sqrt{2 \mu E}}
\end{align}

where $E_q = q^2 / (2 \mu)$.

### Delta identity

In the above equation it was used that
$$
    \int d x \frac{1}{f(x) + i \epsilon} = \mathcal P \int d x \frac{1}{f(x)} - i \pi  \int d x \delta(f(x))
$$
where $\mathcal P$ denotes the principle value integral and
$$
    \int d x \delta(f(x)) g(x) = \sum\limits_{x_i} \frac{g(x_i) }{|f'(x_i)|}
$$
where $x_i$ are the zeros of $f$.

### Basis definitions

The partial wave decomposed matrix element of operators are defined as
$$
    O_{\alpha' \alpha}(k', k)
    =
    \int d \Omega_{k'}  d \Omega_{k}
    Y_{l' m_l'} (\Omega_{k'})
    \left\langle \vec k' , \tilde \alpha' \middle\vert O \middle\vert \vec k, \tilde \alpha \right\rangle
    Y_{l m_l} (\Omega_{k})
$$
with the collection of quantum numbers $\alpha = (\tilde \alpha, l , m_l)$.

More specifically, the angular matrix elements are defined as

\begin{align}
    \left\langle \vec k \middle\vert q l m_l \right\rangle 
    &=
    \frac{\delta(k - q)}{q^2} Y_{l m_l}^*(\Omega_k)
    \, , &
    \mathbb{1}
    &=
    \sum\limits_{l m} \int d q q^2 \left\vert q l m \right\rangle\left\langle q l m \right \vert
\end{align}

In case of $S$-wave only results, this becomes
$$
    O_{0}(k', k) = 
    \frac{1}{4 \pi} 
    \int d \Omega_{k'}  d \Omega_{k}
    \left\langle \vec k' \middle\vert O \middle\vert \vec k \right\rangle
$$

### Simplifications

$$
    \frac{1}{X+i \epsilon}=\mathcal P \frac{1}{X}-i \pi \delta(X)
    \qquad
    \int d x \,  f(x) \delta(g(x)) = \sum_{x_i} \frac{f(x_i)}{|g'(x_i)|}
$$
was used and $g(x_i) = 0$ are all zeros in the integration domain.
The result is 

$$
    \int_{0}^{\infty} d q q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \delta(E-E_{q})
    =
    \frac{\mu}{p_0} p_0^2 \, V_{\alpha'\alpha''}\left(k^{\prime}, p_0\right) T_{\alpha''\alpha}(p_0, k ; E) \bigg|_{p_0 = \sqrt{2 \mu E}}
$$

### Smoothing out

The above equation becomes singular for $E_q = E$.
Eventhough this value is ruled out by the principle value integral, it can be numerically unstable.
For this reason it is replaced by a more stable equivalent.

\begin{multline}
    P \int_{0}^{\infty} d q q^{2}
    \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}}
    \\=
    P \int_{0}^{\infty} d q 
    \frac{q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right)T_{\alpha''\alpha}(q, k ; E) - p_0^2 V_{\alpha'\alpha''}\left(k^{\prime}, p_0 \right)T_{\alpha''\alpha}(p_0, k ; E)  }{E-E_{q}} 
    \\+
    P \int_{0}^{\infty} d q 
    \frac{1}{E-E_{q}} p_0^2 V_{\alpha'\alpha''}\left(k^{\prime}, p_0 \right) T_{\alpha''\alpha}(p_0, k ; E)
\end{multline}
where $p_0 = \sqrt{2 \mu E}$.


The last part is equal to
$$
    P \int_{0}^{\Lambda} d q 
    \frac{1}{E-E_{q}}
    =
    \frac{\mu}{p_0} \log \left(\frac{\Lambda +p_0}{\Lambda -p_0}\right)
    \, , \qquad \Lambda \geq p_0
$$
Note that this integral is zero for infinite cutoff.

Thus, the regularized integral becomes

\begin{multline}
    T_{\alpha'\alpha}\left(k^{\prime}, k ; E\right)
    =
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    + P \int_{0}^{\Lambda} d q 
    \frac{q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right)T_{\alpha''\alpha}(q, k ; E) - p_0^2 V_{\alpha'\alpha''}\left(k^{\prime}, p_0 \right)T_{\alpha''\alpha}(p_0, k ; E)  }{E-E_{q}} 
    \\
    +  \mu p_0 \left[
        \log \left(\frac{\Lambda +p_0}{\Lambda -p_0}\right) -  i \pi
    \right] 
    \sum_{\alpha''}  V_{\alpha'\alpha''}\left(k^{\prime}, p_0\right) T_{\alpha''\alpha}(p_0, k ; E)
\end{multline}

### Discretization

The on-shell equation of the $T$-matrix for $S$-waves only on a numerical grid ($(k_i, w_{k_i})$, $1 \leq i \leq N$) becomes a matrix-vector equation:
\begin{align}
    t_0 &= T_0\left(p_0, p_0, E = \frac{p_0^2}{2 \mu}\right)
    \, , &
    t_i &= T_0\left(k_i, p_0, E = \frac{p_0^2}{2 \mu}\right)
    \, , &
    v_i &= V_0(k_i, p_0)
    \, , &
    v_{i, j} &= V_0(k_i,k_j)
    \, , &
    g_i &= \frac{2 \mu}{p_0^2 - k_i^2}
\end{align}

$$
    t_i 
    = 
    v_i 
    + \sum\limits_{n=1}^N w_{k_n} \left(k_n^2  v_{i n} t_n - p_0^2 v_{i} t_0\right) g_n
    + \mu p_0 \left[\log\left(\frac{\Lambda + p_0}{\Lambda - p_0} \right) -  i \pi  \right] v_i t_0 
    \, , \qquad  1 \leq i \leq N \, .
$$

One can naturally extend this equation by choosing $k_0 = p_0$ and $k_0 \neq k_i \forall i \neq 0$ such that one finds

\begin{align}
    \sum_{n=0}^N A_{in} t_n &= v_i
    \, , &
    A_{in} &=
    \delta_{in} + 
    \begin{cases}
         - w_{k_n} k_n^2  v_{i n} g_n & n > 0  \\
       \mu p_0 \left[
           i \pi  - \log\left(\frac{\Lambda + p_0}{\Lambda - p_0}\right)
           + \frac{p_0}{\mu} \sum\limits_{n=1}^N w_{k_n} g_n  
       \right] v_i 
       & n = 0
    \end{cases}
\end{align}

## Potentials

Choose the Yukawa potential 
$$
    V(\vec p, \vec k) 
    = 
    - \frac{g_A^2}{2 F_\pi^2} 
    \frac{m_\pi^2}{(\vec p - \vec k)^2 + m_\pi^2}
    \leftrightarrow
    V(r) 
    = 
   -\frac{m_{\pi }^2 g_A^2}{8 \pi  F_{\pi }^2} \frac{e^{-m_{\pi }r}}{ r}
$$

$g_A = 1.29$, $f_\pi = 132$MeV and $m_\pi = 141$MeV
The $l = 0$ component is given by



$$
\left\langle\mathbf{k}^{\prime}|\widehat{V}| \mathbf{k}\right\rangle=\frac{2}{\pi} \sum_{l, m} V_{l}\left(k^{\prime}, k\right) Y_{l m}^{*}\left(\Omega_{k^{\prime}}\right) Y_{l m}\left(\Omega_{k}\right)
\qquad
V_{l}\left(k^{\prime}, k\right)=\int_{0}^{\infty} r^{2} d r \, j_{l}(k r) V(r) j_{l}\left(k^{\prime} r\right)
$$
adn thus
$$
 V_0(k, k') = \frac{m_{\pi }^2 g_A^2 }{32 \pi 
   F_{\pi }^2 } \frac{1}{k k'}\log \left(1-\frac{4 k k'}{(k+k')^2+m_{\pi }^2}\right)
$$

The partial wave decomposed matrix element of operators are defined as
$$
    O_{\alpha' \alpha}(k', k)
    =
    \int d \Omega_{k'}  d \Omega_{k}
    Y_{l' m_l'} (\Omega_{k'})
    \left\langle \vec k' , \tilde \alpha' \middle\vert O \middle\vert \vec k, \tilde \alpha \right\rangle
    Y_{l m_l} (\Omega_{k})
$$
with the collection of quantum numbers $\alpha = (\tilde \alpha, l , m_l)$.

More specifically, the angular matrix elements are defined as

\begin{align}
    \left\langle \vec k \middle\vert q l m_l \right\rangle 
    &=
    \frac{\delta(k - q)}{q^2} Y_{l m_l}^*(\Omega_k)
    \, , &
    \mathbb{1}
    &=
    \sum\limits_{l m} \int d q q^2 \left\vert q l m \right\rangle\left\langle q l m \right \vert
\end{align}

In case of $S$-wave only results, this becomes
$$
    O_{0}(k', k) = 
    \frac{1}{4 \pi} 
    \int d \Omega_{k'}  d \Omega_{k}
    \left\langle \vec k' \middle\vert O \middle\vert \vec k \right\rangle
$$

In [ ]:
def V1(k, mpi=141., gA=1.29, fpi=132.):
    kk = k.reshape(-1, 1)
    return gA**2 * mpi**2/ 32 / np.pi / fpi**2 / kk / k * np.log(1 - 4 * kk * k / ((k+kk)**2 + mpi**2)) 

def V(k):
    return np.ones([k.size]*2)

## Computing T

In [ ]:
p0 = [(pp + p[n+1])/2 for n, pp in enumerate(p[:-1])][:10]


phase_shifts = []

for pp0 in p0:
    pp = np.append([pp0], p)
    v = -V(pp)
    E = pp**2 / 2 / mu

    A = np.zeros([pp.size]*2, dtype=complex)
    
    gint = np.sum(wp / (E[0] - E[1:]))
    
    print(- np.log((pp[-1] + pp0) / (pp[-1] - pp0)) * mu / pp0 + gint)
    
    for i, n in product(*[range(pp.size), range(pp.size)]):
        
        if i == n:
            A[i, i] = 1
        
        if n == 0:
            A[i, n] += mu * pp0 * ( 
                1j * np.pi 
                - np.log((pp[-1] + pp0) / (pp[-1] - pp0))
                + gint * pp0 / mu
            )* v[i, 0]
        else:
            A[i, n] -= wp[n-1] * p[n-1]**2 * v[i, n] / (E[0] - E[n])
            
    t = np.linalg.solve(A , np.diagonal(v))
    
    f = mu * t / 2 / np.pi

    ere = 1 / f + 1j * pp0
    
    phase_shifts.append(ere[0])
    
phase_shifts = np.array(phase_shifts)

### Result

$$
    C_{0}(\Lambda)=\frac{2 \pi / \mu}{1 / a_{0}-2 \Lambda / \pi}
$$

In [ ]:
1/t.real

In [ ]:
c0 = - V(np.array([1])).mean()
Lambda = p[-1]
a0 =  1 / (2 * np.pi / mu / c0 + 2 * Lambda / np.pi)

ere.real.mean() * a0 

In [ ]:
fig, ax = plt.subplots()

ax.plot(p0, phase_shifts.real, ".", label="Re")
ax.plot(p0, phase_shifts.imag, "--", label="Im")

#ax.axhline(-a0inv, lw=5)


In [ ]:
- 1 /  / np.pi

In [ ]:
A.real

In [ ]:
fig, ax = plt.subplots()

ax.plot(pp[1:], A[0, 1:].imag, ".")
ax.axvline(pp[0])

ax.set_ylim(-10, 10)

plt.show()

In [ ]:
plt.plot(p0, phase_shifts.imag, ".")

In [ ]:
(1 / t).real

Using
\begin{align}
    T_{l}\left(k^{\prime}, k ; E\right)
    &=
    V_{l}\left(k^{\prime}, k\right)+\frac{2}{\pi} 
    \int_{0}^{\infty} d q q^{2} 
    \frac{V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E)}{E-E_{q}+i \epsilon}
    \\
    &=
    V_{l}\left(k^{\prime}, k\right)
    +\frac{2}{\pi} \mathcal P \int_{0}^{\infty} d q q^{2} 
    \frac{V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E)}{E-E_{q}}
    - 2 i \int_{0}^{\infty} d q q^{2} V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \delta(E-E_{q})
\end{align}
where
$$
    \frac{1}{X+i \epsilon}=\mathcal P \frac{1}{X}-i \pi \delta(X)
    \qquad
    \int d x \,  f(x) \delta(g(x)) = \sum_{x_i} \frac{f(x_i)}{|g'(x_i)|}
$$
was used and $g(x_i) = 0$ are all zeros in the integration domain.
The result is 

$$
    \int_{0}^{\infty} d q q^{2} V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \delta(E-E_{q})
    =
    \frac{\mu}{q} q^2 \, V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \bigg|_{q = \sqrt{2 \mu E}}
$$

We can write the on shell $T$ matrix equation as
$$
    \vec t = \vec v + M \vec t \Rightarrow (1 - M) \vec t = \vec v
$$
with 
\begin{align}
    t_i &= T_0(k_i, k_i, E_{k_i})
    \, , &
    v_i &= V_0(k_i, k_i, E_{k_i})
    \, , &
    M_{ij} &=
    \begin{cases}
        \frac{2}{\pi} k_j^2 w_{k_j} \frac{V_0(k_i, k_j)}{E_{k_i} - E_{k_j}} & i \neq j \\
        - 2 i \, \mu k_i \, V_{0}\left(k_i, k_i\right) & i = j
    \end{cases}
\end{align}

In [ ]:
p

In [ ]:
p0

In [ ]:
A

In [ ]:
2.1415926535892003 / np.pi

In [ ]:
plt.contourf(A[1:,1:].real)
plt.colorbar()

In [ ]:
plt.contourf(A[1:,1:].imag)
plt.colorbar()

In [ ]:
len(1)

In [ ]:
np.set_printoptions(precision=2)
A

In [ ]:
plt.plot(p, 1/ (np.linalg.inv(A)@np.diag(vpp)).real, "o")

In [ ]:


ere.imag

In momentum space, the potential is given by 
$$
    \left\langle \vec p | V | \psi \right\rangle 
    =
    \int \frac{d \vec k^3}{(2\pi)^3} V(\vec p, \vec k) \psi(\vec k)
$$
where in the last case, it was used that the angular momentum is zero ($l=0$).

In [ ]:
(1/f).real

In [ ]:
A@t - np.diagonal(vpp)

In [ ]:
vpp

In [ ]:
vpp - vpp.T

In [ ]:
A = np.eye(M.shape[0]) - M

In [ ]:
M = 2 / np.pi * p**2 * wp * V(p) / Ginv + 0j
for i in range(M.shape[0]):
    M[i, i] = -2j * mu * p[i] * v[i]

In [ ]:
M.real

In [ ]:
M.real

In [ ]:
1/ t.imag 

In [ ]:
p